# 简单全连接神经网络

In [2]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='Nadam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=10)
_, acc=model.evaluate(x_test,  y_test, verbose=2)
print("acc:%.3f"%(acc))


Epoch 1/10
6000/6000 [==============================] - 6s 997us/step - loss: 0.4591 - accuracy: 0.8609
Epoch 2/10
6000/6000 [==============================] - 6s 985us/step - loss: 0.3113 - accuracy: 0.9071
Epoch 3/10
6000/6000 [==============================] - 6s 971us/step - loss: 0.2795 - accuracy: 0.9144
Epoch 4/10
6000/6000 [==============================] - 6s 989us/step - loss: 0.2610 - accuracy: 0.9203
Epoch 5/10
6000/6000 [==============================] - 6s 997us/step - loss: 0.2478 - accuracy: 0.9248
Epoch 6/10
6000/6000 [==============================] - 6s 970us/step - loss: 0.2325 - accuracy: 0.9279
Epoch 7/10
6000/6000 [==============================] - 6s 1ms/step - loss: 0.2327 - accuracy: 0.9284
Epoch 8/10
6000/6000 [==============================] - 7s 1ms/step - loss: 0.2302 - accuracy: 0.9298
Epoch 9/10
6000/6000 [==============================] - 6s 970us/step - loss: 0.2218 - accuracy: 0.9314
Epoch 10/10
6000/6000 [==============================] - 6s 981us/st

# CNN识别手写数字

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
n,m = x_train.shape[1], x_train.shape[2]
model = Sequential()
model.add(Reshape((n,m,1), input_shape=(n,m)))
model.add(Conv2D(filters=32, kernel_size=5, activation='relu', padding="same"))
model.add(MaxPool2D())
model.add(Conv2D(filters=64, kernel_size=(5,5), activation='relu', padding="same"))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(1024,activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='Nadam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=100)
model.evaluate(x_test,  y_test, verbose=2)

for layer in model.layers:
    print(('%s - %s')%(layer.input_shape, layer.output_shape))

Epoch 1/10


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


600/600 [==============================] - 51s 85ms/step - loss: 0.1387 - accuracy: 0.9562
Epoch 2/10
600/600 [==============================] - 51s 84ms/step - loss: 0.0425 - accuracy: 0.9872
Epoch 3/10
600/600 [==============================] - 51s 84ms/step - loss: 0.0300 - accuracy: 0.9909
Epoch 4/10
600/600 [==============================] - 51s 85ms/step - loss: 0.0243 - accuracy: 0.9921
Epoch 5/10
600/600 [==============================] - 51s 84ms/step - loss: 0.0179 - accuracy: 0.9941
Epoch 6/10
600/600 [==============================] - 51s 84ms/step - loss: 0.0145 - accuracy: 0.9953
Epoch 7/10
600/600 [==============================] - 51s 84ms/step - loss: 0.0127 - accuracy: 0.9959
Epoch 8/10
600/600 [==============================] - 51s 84ms/step - loss: 0.0107 - accuracy: 0.9967
Epoch 9/10
600/600 [==============================] - 50s 83ms/step - loss: 0.0105 - accuracy: 0.9967
Epoch 10/10
600/600 [==============================] - 50s 83ms/step - loss: 0.0079 - accurac

# resnet手写数字识别

In [1]:
import torch
torch.__version__

'1.11.0+cpu'

# 数据预处理

In [2]:
from torchvision.models import resnet18
from torch import nn
from torch.utils.data import DataLoader

In [3]:
model = resnet18(num_classes=10)

In [4]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
train_ds = MNIST("mnist", train=True, download=True, transform=ToTensor())
test_ds = MNIST("mnist", train=False, download=True, transform=ToTensor())
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=64)

In [10]:
import pytorch_lightning as pl
class ResNetMNIST(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = resnet18(num_classes=10)
    self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    self.loss = nn.CrossEntropyLoss()

  def forward(self, x):
    return self.model(x)
  
  def training_step(self, batch, batch_no):
    x, y = batch
    logits = self(x)
    loss = self.loss(logits, y)
    return loss
  
  def configure_optimizers(self):
    return torch.optim.RMSprop(self.parameters(), lr=0.005)

# 训练

In [13]:
model = ResNetMNIST()
trainer = pl.Trainer(
    gpus=0,
    max_epochs=1,
    progress_bar_refresh_rate=20
)

C:\ProgramData\Anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [14]:
trainer.fit(model, train_dl)

Missing logger folder: C:\Users\422\深度学习\lightning_logs

  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 11.2 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.701    Total estimated model params size (MB)
C:\ProgramData\Anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


In [15]:
trainer.save_checkpoint("resnet18_mnist.pt")

In [16]:
def get_prediction(x, model: pl.LightningModule):
  model.freeze() # prepares model for predicting
  probabilities = torch.softmax(model(x), dim=1)
  predicted_class = torch.argmax(probabilities, dim=1)
  return predicted_class, probabilities

In [17]:
from tqdm.autonotebook import tqdm

In [18]:
inference_model = ResNetMNIST.load_from_checkpoint("resnet18_mnist.pt")